In [1]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import pandas as pd

# files
chi_2020_2030_file = 'chi_vals_2020-12-01_2030-11-30.txt'
land_mask_file = 'land_mask_ind_2020-12-01_2030-11-30.txt'
rlat_file = 'lat_coords_2020-12-01_2030-11-30.txt'
rlon_file = 'lon_coords_2020-12-01_2030-11-30.txt'

land_mask = pd.read_csv(land_mask_file, delimiter=" ")
rlats = pd.read_csv(rlat_file, delimiter=" ", header=None)
rlons = pd.read_csv(rlon_file, delimiter=" ", header=None)
chi = pd.read_csv(chi_2020_2030_file, delimiter = " ", header=None)
#chi
#land_mask
rlats
#rlons.head(20)

,0
0,-0.934999
1,-0.824999
2,-0.714998
3,-0.605000
4,-0.494999
...,...
123,12.595001
124,12.705002
125,12.815002
126,12.925003


In [2]:
# take all chi values for one reference point as an example
all_corr = chi.iloc[0]
# find the coordinates of this example point
ref_rlat, ref_rlon = rlats.iloc[land_mask.iloc[0].row][0], rlons.iloc[land_mask.iloc[0].col][0]

# find coordinates of each correlated point
rlat_corr = []
rlon_corr = []
for i in range(all_corr.size):
    rlat_corr.append(rlats.iloc[land_mask.iloc[i].col][0])
    rlon_corr.append(rlons.iloc[land_mask.iloc[i].row][0])

In [3]:
# transform from rotated to regular coordinates
from transform_coordinates import unrotate_rll
lats, lons = unrotate_rll(np.asarray(rlat_corr), np.asarray(rlon_corr))
pd.DataFrame(data={'chi': all_corr, 'lat': lats, 'lons': lons}).to_csv('test.csv')

In [12]:
# convert to xarray for neat coordinate visualisation
idx = pd.MultiIndex.from_arrays(arrays=[lats,lons], names=["lat","lon"])
s = pd.Series(data=all_corr.tolist(), index=idx)
# use from_series method
da = xr.DataArray.from_series(s)

lat        lon      
49.286571  -1.958907    1.000000
49.392650  -2.003581    0.866667
49.545760  -2.420388    0.872222
50.924787   0.393959    0.766667
50.789611  -1.168894    0.766667
                          ...   
60.577351  -1.159222    0.372222
60.683587  -1.217399    0.361111
60.711479  -1.004356    0.400000
60.845438  -0.848436    0.394444
61.695104  -6.779161    0.383333
Length: 2064, dtype: float64

In [ ]:
# TODO: ready to be plotted on simple map